In [86]:
import email, glob
import pandas as pd
from flanker import mime
import string
import re
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from pprint import pprint
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [87]:
def remove_content_in_braces(msg) :
	msg1 = ''
	cnt = 0
	for char in msg :
		if char == '{' :
			cnt += 1
		elif char == '}' :
			cnt -= 1
		elif cnt == 0 :
			msg1 += char
		else :
			continue
	return msg1

def remove_func_and_struct(msg) :
	msg1 = ''
	take_line = True
	msg = msg.splitlines()
	for line in msg :
		take_line = True
		if line == '' :
			continue
		words = line.split(' ')
		if words[0] == "func" :
			take_line = False
		elif words[0] == "type" :
			if len(words)  >= 3  and words[2] == "struct" :
				take_line = False
		if take_line :
			msg1 += (line + '\n')
	return msg1

def remove_other_code_lines(msg) :
	msg1 = ''
	take_line = True
	msg = msg.splitlines()
	i = 0	
	while i < len(msg) :
		if (msg[i] == '') or ("//" in msg[i]) :
			i += 1
			continue
		take_line = True
		line = msg[i]
		if "package" in line :
			words = line.split(' ')
			if len(words) < 4 :
				take_line = False
		elif "import" in line :
			words = line.split(' ')
			if len(words) < 4 :
				take_line = False
				if "(" in line :
					while ')' not in msg[i] :
						i += 1
		elif "const" in line :
			words = line.split(' ')
			if len(words) < 4 :
				take_line = False
				if "(" in line :
					while ')' not in msg[i] :
						i += 1
		if take_line :
			msg1 += line + '\n'
		i += 1
	return msg1


def remove_code(msg) :
	msg = (remove_content_in_braces(msg))
	msg = (remove_func_and_struct(msg))
	msg = (remove_other_code_lines(msg))
	return msg

def get_header(msg):
    msg = email.message_from_string(msg)
    mfrom = msg['From'].split('<')[0]
    return mfrom
    
def flan(msg):
    rt = ''
    msg = mime.from_string(msg)
    if msg.content_type.is_singlepart():
      temp = str(msg.body)
      temp = temp.splitlines()
      for _ in temp:
          if _.startswith('>'):
              continue
          elif _.startswith('On'):
              continue
          else:
              rt+=_+"\n"
    else :
      for part in msg.parts :
          if "(text/plain)" in str(part) :
              temp = str(part.body)
              temp = temp.splitlines()
              for _ in temp :
                  if _.startswith('>') :
                      continue
                  if _.startswith('On'):
                      continue
                  else :
                      rt+=_+"\n"
    return rt

In [88]:
rt =''
fpath = "/home/parth/BE_Project/EmailRecommmendation/features/Dataset/*/*.email"
files = glob.glob(fpath)
for file in files :
  f = open(file, "r")
  msg = f.read()
  msg = mime.from_string(msg)
  if msg.content_type.is_singlepart():
      temp = str(msg.body)
      temp = temp.splitlines()
      for _ in temp:
          if _.startswith('>'):
              continue
          elif _.startswith('On'):
              continue
          else:
              rt+=_+"\n"
  else :
      for part in msg.parts :
          if "(text/plain)" in str(part) :
              temp = str(part.body)
              temp = temp.splitlines()
              for _ in temp :
                  if _.startswith('>') :
                      continue
                  if _.startswith('On'):
                      continue
                  else :
                      rt+=_+"\n"
               
rt = remove_code(rt)
# print(rt)
rt = rt.split('\n')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(rt)
total_words = len(tokenizer.word_index) + 1
max_words=total_words

In [89]:
df = pd.DataFrame(columns=['body','replier', 'thread_no'])
users = []
fpath = "/home/parth/BE_Project/EmailRecommmendation/features/Dataset/*"
folder = glob.glob(fpath)
th_no = 0

for fol in folder:
    files = glob.glob(fol+'/*.email')
    flag = 0
    t = ''
    for file in files:
        if flag==0:
            data = open(file,'r')
            temp = data.read()
            header = get_header(temp)
            temp = flan(temp)
            temp = remove_code(temp)
            t = temp
            flag = 1
            users.append(header)
            
            continue
        data = open(file,'r')
        temp = data.read()
        header = get_header(temp)
        users.append(header)
        temp = flan(temp)
        temp = remove_code(temp)
        df = df.append({'body': t,'replier':header, 'thread_no':th_no}, ignore_index=True)
        t = t + temp
    th_no += 1

In [90]:
print(df.head)

<bound method NDFrame.head of                                                  body  \
0   Not at all. I was just trying to reduce the nu...   
1   Not at all. I was just trying to reduce the nu...   
2   Not at all. I was just trying to reduce the nu...   
3   Not at all. I was just trying to reduce the nu...   
4   Not at all. I was just trying to reduce the nu...   
5   Not at all. I was just trying to reduce the nu...   
6   Not at all. I was just trying to reduce the nu...   
7   Not at all. I was just trying to reduce the nu...   
8   A stupid question, but I use it quite often in...   
9   A stupid question, but I use it quite often in...   
10  A stupid question, but I use it quite often in...   
11  A stupid question, but I use it quite often in...   
12  Be "go language" going to support  android ui ...   
13  Actually, there are more problems than those, ...   
14  Actually, there are more problems than those, ...   
15  Actually, there are more problems than those, ...   
1

In [91]:
h = df.replier
pprint(len(h))
h=list(h)

thread_no_list = df.thread_no
thread_no_list = list(thread_no_list)
print(thread_no_list)

68
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 10, 10, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 15, 15, 16, 16, 16, 16, 16]


In [129]:
np.set_printoptions(threshold = sys.maxsize)
val = np.array(users)
w = open('one_hot.txt','w')

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(val)
print(integer_encoded)
user_indices = integer_encoded

one_hot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded),1)
one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
output_shape = one_hot_encoded.shape[1]
w.write(str(one_hot_encoded))
w.close()

[22 26 28 23 20 22 16 22 22 37 25 13 29  8  2 24 21  5  5 17 21 36 30 13
  3 25 37 25 37 37 18  4 18  4 18 18 15 25 25 11 24  0 29 10  0 11 31 33
  1  6 37 33 26 33 20 14 14 34 31 33 26  7  9  6  6 37 12 12  6 25 19 13
  6  6  6 35 20 10 27  1 13 37  1  1 25 32]


/home/parth/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [128]:
print(user_indices)
input_weights = []
user_size = max(user_indices) + 1
print(user_size)

[22 26 28 23 20 22 16 22 22 37 25 13 29  8  2 24 21  5  5 17 21 36 30 13
  3 25 37 25 37 37 18  4 18  4 18 18 15 25 25 11 24  0 29 10  0 11 31 33
  1  6 37 33 26 33 20 14 14 34 31 33 26  7  9  6  6 37 12 12  6 25 19 13
  6  6  6 35 20 10 27  1 13 37  1  1 25 32]
38


In [113]:
onehot_encoded_final = []
for replier in h:
    idx1 = label_encoder.transform([replier])
    onehot = np.zeros(user_size)
    onehot[idx1] = 1
    onehot_encoded_final.append(list(onehot))
one_hot_encoded_final = np.array(onehot_encoded_final)
print(type(one_hot_encoded_final))

<class 'numpy.ndarray'>


In [133]:
#Binary encoding of users participating in each thread

index=0
weight_list = []
for i in range(0, max(thread_no_list)+1):
#     temp = np.zeros(user_size)
    temp_index=index
#     print(temp)
    array  = np.zeros(user_size)
    for j in range(temp_index, temp_index + thread_no_list.count(i)):
#         print(user_indices[j], end = ",")
        array[user_indices[j]] += 1
        weight_list.append(list(array))
#         temp[user_indices[j]] += 1
        index+=1
#     print("\n")
#     print(temp,"\n")

weights = np.array(weight_list)
pprint(weights.shape) 

(68, 38)


In [114]:
x_train = df.body
# print(x_train)

In [115]:
def longest(l):
    m=0
    for k in l:
        m = max(len(k),m)
    return m
max_len = longest(x_train)

In [116]:
# max_words = 1294
# max_len = 3267
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x_train)
sequences = tok.texts_to_sequences(x_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [130]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(output_shape,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [131]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 2843)              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 2843, 50)          64700     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_9 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 38)                9766      
__________

In [132]:
model.fit(sequences_matrix,one_hot_encoded_final,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 54 samples, validate on 14 samples
Epoch 1/10
54/54 [==============================] - 3s 62ms/step - loss: 0.6929 - acc: 0.5404 - val_loss: 0.6874 - val_acc: 0.9380
Epoch 2/10
54/54 [==============================] - 2s 41ms/step - loss: 0.6871 - acc: 0.8879 - val_loss: 0.6788 - val_acc: 0.9568
Epoch 3/10
54/54 [==============================] - 2s 41ms/step - loss: 0.6782 - acc: 0.9176 - val_loss: 0.6613 - val_acc: 0.9737
Epoch 4/10
54/54 [==============================] - 2s 45ms/step - loss: 0.6584 - acc: 0.9410 - val_loss: 0.6116 - val_acc: 0.9737
Epoch 5/10
54/54 [==============================] - 3s 47ms/step - loss: 0.6024 - acc: 0.9469 - val_loss: 0.3638 - val_acc: 0.9737
Epoch 6/10
54/54 [==============================] - 3s 47ms/step - loss: 0.3654 - acc: 0.9552 - val_loss: 0.1753 - val_acc: 0.9737
Epoch 7/10
54/54 [==============================] - 3s 51ms/step - loss: 0.1900 - acc: 0.9722 - val_loss: 0.1425 - val_acc: 0.9737
Epoch 8/10
54/54 [=====================